In [892]:
# Import basic libraries
import numpy as np
import pandas as pd

In [893]:
# import both the datasets for this 
movies_data = pd.read_csv("../Datasets/Prj-MovieRecommender/tmdb_5000_movies.csv")
credits = pd.read_csv("../Datasets/Prj-MovieRecommender/tmdb_5000_credits.csv")

In [894]:
movies_data.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [895]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [896]:
# try reading 'genres' to find names for it...A test
import json
recs = movies_data.head(2)
genre = json.loads(recs['genres'][0])
for gen in genre:
    print(gen['name'])

Action
Adventure
Fantasy
Science Fiction


In [897]:
# try reading 'casts' to find characters and artist for it
# ...A test
casts = credits.head(1)['cast'][0]
cast = json.loads(casts)
for ca in cast:
    print('Character  : ', ca['character'])
    print('Played by  : ', ca['name'], '\n')

Character  :  Jake Sully
Played by  :  Sam Worthington 

Character  :  Neytiri
Played by  :  Zoe Saldana 

Character  :  Dr. Grace Augustine
Played by  :  Sigourney Weaver 

Character  :  Col. Quaritch
Played by  :  Stephen Lang 

Character  :  Trudy Chacon
Played by  :  Michelle Rodriguez 

Character  :  Selfridge
Played by  :  Giovanni Ribisi 

Character  :  Norm Spellman
Played by  :  Joel David Moore 

Character  :  Moat
Played by  :  CCH Pounder 

Character  :  Eytukan
Played by  :  Wes Studi 

Character  :  Tsu'Tey
Played by  :  Laz Alonso 

Character  :  Dr. Max Patel
Played by  :  Dileep Rao 

Character  :  Lyle Wainfleet
Played by  :  Matt Gerald 

Character  :  Private Fike
Played by  :  Sean Anthony Moran 

Character  :  Cryo Vault Med Tech
Played by  :  Jason Whyte 

Character  :  Venture Star Crew Chief
Played by  :  Scott Lawrence 

Character  :  Lock Up Trooper
Played by  :  Kelly Kilgour 

Character  :  Shuttle Pilot
Played by  :  James Patrick Pitt 

Character  :  Shut

In [898]:
# try reading 'crew' to find department, job and the name of the person doing the job
# ...A test
crews = credits.head(1)['crew'][0]
crew = json.loads(crews)
for ca in crew:
    print('\nDepartment :', ca['department'] )
    print( 'Job        :', ca['job'])
    print( 'Name       :', ca['name'])


Department : Editing
Job        : Editor
Name       : Stephen E. Rivkin

Department : Art
Job        : Production Design
Name       : Rick Carter

Department : Sound
Job        : Sound Designer
Name       : Christopher Boyes

Department : Sound
Job        : Supervising Sound Editor
Name       : Christopher Boyes

Department : Production
Job        : Casting
Name       : Mali Finn

Department : Sound
Job        : Original Music Composer
Name       : James Horner

Department : Directing
Job        : Director
Name       : James Cameron

Department : Writing
Job        : Writer
Name       : James Cameron

Department : Editing
Job        : Editor
Name       : James Cameron

Department : Production
Job        : Producer
Name       : James Cameron

Department : Writing
Job        : Screenplay
Name       : James Cameron

Department : Art
Job        : Art Direction
Name       : Andrew Menzies

Department : Visual Effects
Job        : Visual Effects Producer
Name       : Jill Brooks

Department

In [899]:
# check if titles are non-unique. Will use it to merge if they are unique
sum(credits.title.value_counts() > 1)
# oops...!  'title' has duplicacy. Hence I will not use it to merge two data-frames
# A better approiach will be to drop duplicate records. Records having same title

3

In [900]:
# check those cases where 'title' in movies_data is not equal to 'title' in credits
movies_data['title'].value_counts()

The Host                 2
Batman                   2
Out of the Blue          2
Avatar                   1
The Girl on the Train    1
                        ..
Step Up 3D               1
Secondhand Lions         1
The Age of Adaline       1
Drag Me to Hell          1
My Date with Drew        1
Name: title, Length: 4800, dtype: int64

In [901]:
credits['title'].value_counts()

The Host                 2
Batman                   2
Out of the Blue          2
Avatar                   1
The Girl on the Train    1
                        ..
Step Up 3D               1
Secondhand Lions         1
The Age of Adaline       1
Drag Me to Hell          1
My Date with Drew        1
Name: title, Length: 4800, dtype: int64

In [902]:
# Merge two dataframes based on the 'id' and 'movie_id'
# if I make 'id' as index of the dataframe, it will simplify things in future
df = movies_data.merge(credits, left_on='id', right_on='movie_id')
#df = movies.merge(credits, on='title')
df.shape

# looks good as number of records are not increased or decreased. 
# 'id' and 'movie_id' are same value but I will deal with it during feature selection

(4803, 24)

In [903]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew'],
      dtype='object')

In [904]:
# drom 'movie_id' as it is same as 'id'
# drop 'title_y' column and chnage name of the 'title_x' to just 'title'
movies = df.drop(['movie_id', 'title_y'], axis=1)
movies = movies.rename(columns={'title_x': 'title'})
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [905]:
# drop the duplicate records (although they have unique 'id' but everytrhing else is same.)
movies = movies.drop_duplicates(subset=['title'])
movies.shape

(4800, 22)

In [906]:
# check total columns in the data-frame
print("Columns : ", movies.columns)
movies.info()

Columns :  Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4800 non-null   int64  
 1   genres                4800 non-null   object 
 2   homepage              1711 non-null   object 
 3   id                    4800 non-null   int64  
 4   keywords              4800 non-null   object 
 5   original_language     4800 non-null   object 
 6   original_title        4800 non-null   object 
 7   overview              4797 non-null   object 
 8   popularity  

In [907]:
# status of the movie is mostly release. Highly imbalances dataset
# hence will ignore this columns
movies.status.value_counts()

Released           4792
Rumored               5
Post Production       3
Name: status, dtype: int64

<!-- ## Here we are skipping numerical values columns as NLP and text based algorithms are considered
##
##

# columns to keep (these column seems to make sense in decision making)
#
# ['genres'                 : important as it tells the flavour and theme
# 'id' (Ignore-Later)       : Not useful in model training. Will be meaningful in identifying related information later
# 'keywords'                : Tag for the movie. Very important information
# 'overview'                : Will define plot and description keywords. Will be helpful.
# 'title'                   : May not be important but let us keep it.
# 'cast'                    : Very important 
# 'crew'                    : Very imporatant
# ]
#

# Columns to drop as they do not appear to e useful in our deecision making
#
# 'budget' (Later)           : Should be useful but for now ignore it. Budget means resources used in creating a movie. More resources more mature graphics, locations, actors fee, set quality etc
# 'original_language'        : Most of the movies are in english. So let us drop it as it does not contribute to the variability in the dataset
#                              movies[].value_counts() 
# 'original_title'           : it may be in non-english hence cause issues. Will not add much value.
# 'popularity' (Later)       : people do watch mivies by popularity. However it can be ignored in our problem solving approach.
# 'production_companies' (Later): Should be useful but ignore for now
# 'production_countries' (Later): We do care for countries of origin. But not the shooting places. It's ambiguous.
# 'release_date' (Later)     : Time of release does matter
# 'revenue' (Later)          : Revenue earned may be indicator of how many people liked it. Despite individuals choices.
#                              Though it may be meaningless if we want to cater specific class of people. Mobocracy may not work :-) 
# 'runtime'                  : Duration should not matter   
# 'spoken_languages'         : Should not matter
# 'status'                   : status is mostly 'release'. Highly imbalances dataset, hence will ignore this columns.
# 'tagline'                  : tagline does not matte. It may be totally irrelevant. Does not convey reson to cause of liking
# 'vote_average'   (Later)   : Valuabe but numeric       
# 'vote_count'   (Later)     : Valuabe but numeric. will re-evaluate as common people tend to rate very-good and very-bad movies only    
# 'homepage'] -->

In [908]:
## Here we are skipping numerical values columns as NLP and text based algorithms are considered
##
##

# columns to keep (these column seems to make sense in decision making)
#
# ['genres'                 : important as it tells the flavour and theme
# 'id' (Ignore-Later)       : Not useful in model training. Will be meaningful in identifying related information later
# 'keywords'                : Tag for the movie. Very important information
# 'overview'                : Will define plot and description keywords. Will be helpful.
# 'title'                   : May not be important but let us keep it.
# 'cast'                    : Very important 
# 'crew'                    : Very imporatant
# ]
#

# Columns to drop as they do not appear to e useful in our deecision making
#
# 'budget' (Later)           : Should be useful but for now ignore it. Budget means resources used in creating a movie. More resources more mature graphics, locations, actors fee, set quality etc
# 'original_language'        : Most of the movies are in english. So let us drop it as it does not contribute to the variability in the dataset
#                              movies[].value_counts() 
# 'original_title'           : it may be in non-english hence cause issues. Will not add much value.
# 'popularity' (Later)       : people do watch mivies by popularity. However it can be ignored in our problem solving approach.
# 'production_companies' (Later): Should be useful but ignore for now
# 'production_countries' (Later): We do care for countries of origin. But not the shooting places. It's ambiguous.
# 'release_date' (Later)     : Time of release does matter
# 'revenue' (Later)          : Revenue earned may be indicator of how many people liked it. Despite individuals choices.
#                              Though it may be meaningless if we want to cater specific class of people. Mobocracy may not work :-) 
# 'runtime'                  : Duration should not matter   
# 'spoken_languages'         : Should not matter
# 'status'                   : status is mostly 'release'. Highly imbalances dataset, hence will ignore this columns.
# 'tagline'                  : tagline does not matte. It may be totally irrelevant. Does not convey reson to cause of liking
# 'vote_average'   (Later)   : Valuabe but numeric       
# 'vote_count'   (Later)     : Valuabe but numeric. will re-evaluate as common people tend to rate very-good and very-bad movies only    
# 'homepage']

In [909]:
# keep the columns of interest only
movies = movies[['id', 'title', 'overview', 'genres', 'keywords',  'cast', 'crew']]
movies.sample(2)

,id,title,overview,genres,keywords,cast,crew
2848,26710,My Baby's Daddy,A trio of young men are forced to grow up quic...,"[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 17985, ""name"": ""birth""}, {""id"": 160946...","[{""cast_id"": 1, ""character"": ""Lonnie"", ""credit...","[{""credit_id"": ""53d5f724c3a3686b880054fe"", ""de..."
4413,335874,Childless,Katherine is a typical teenager. Today's her f...,[],[],"[{""cast_id"": 1, ""character"": """", ""credit_id"": ...","[{""credit_id"": ""55575bb49251411e62005cfd"", ""de..."


In [910]:
# check null values in the dataframe of interest
# Looks like I need to drop 3 records due to overview being null for them 
movies.isnull().sum(axis=0)

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [911]:
# dropped row with missing overview...but thats not required in fact. It has other information which is valuable
movies = movies.dropna(axis=0)
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4797 entries, 0 to 4802
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4797 non-null   int64 
 1   title     4797 non-null   object
 2   overview  4797 non-null   object
 3   genres    4797 non-null   object
 4   keywords  4797 non-null   object
 5   cast      4797 non-null   object
 6   crew      4797 non-null   object
dtypes: int64(1), object(6)
memory usage: 299.8+ KB


In [912]:
# function to read a column from record and return values list based on the keys list
import json
def getvalues(mylist, keys=[]):
    val_list = []
    lst = json.loads(mylist)
    counter = 0
    for elem in lst:
        if counter < 3:
            for key in keys:            
                val_list.append(elem[key])
            counter = counter + 1
        else:
            break
    return val_list

In [913]:
# make generes words into a list and replace in the column
movies['genres'] = movies['genres'].apply(getvalues, keys=['name'])
# make keywords words into a list and replace in the column
movies['keywords'] = movies['keywords'].apply(getvalues, keys=['name'])
# make cast words into a list and replace in the column
movies['cast'] = movies['cast'].apply(getvalues, keys=['name'])

In [914]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy]","[culture clash, future, space war]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island]","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent]","[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama]","[dc comics, crime fighter, terrorist]","[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion]","[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [915]:
# pick the director name from the crew list
import json
def getdirector(mylist):
    val_list = []
    lst = json.loads(mylist)
    for elem in lst:
        if elem['job'] == 'Director':
            val_list.append(elem['name'])
            break;
    return val_list

In [916]:
movies['crew'] = movies['crew'].apply(getdirector)

In [917]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [918]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy]","[culture clash, future, space war]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island]","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama]","[dc comics, crime fighter, terrorist]","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion]","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [919]:
# remove blanks as name should be uniques in entirety
movies['overview'] = movies['overview'].apply(lambda st: [word.replace(" ", "") for word in st])
movies['genres'] = movies['genres'].apply(lambda st: [word.replace(" ", "") for word in st])
movies['keywords'] = movies['keywords'].apply(lambda st: [word.replace(" ", "") for word in st])
movies['cast'] = movies['cast'].apply(lambda st: [word.replace(" ", "") for word in st])
movies['crew'] = movies['crew'].apply(lambda st: [word.replace(" ", "") for word in st])

In [920]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [921]:
movies.drop(['overview', 'genres', 'keywords', 'cast', 'crew'], axis=1, inplace=True)

In [922]:
# avoid issues caused by just the case of the letters
movies['title'] = movies['title'].apply(str.lower)

In [923]:
movies.head()

,id,title,tags
0,19995,avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,pirates of the caribbean: at world's end,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,the dark knight rises,"[Following, the, death, of, District, Attorney..."
4,49529,john carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [924]:
# convert the 'tags' list into a string (paragraph)
movies['tags'] = movies['tags'].apply(lambda mylist: ' '.join(mylist))
# convert the text into lowercase to avoid bigger vector size due to case issues
movies['tags'] = movies['tags'].apply(str.lower)

In [925]:
movies.head()

,id,title,tags
0,19995,avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha..."
2,206647,spectre,a cryptic message from bond’s past sends him o...
3,49026,the dark knight rises,following the death of district attorney harve...
4,49529,john carter,"john carter is a war-weary, former military ca..."


In [926]:
## Planning 
# Remove StopWords
# Apply Stemming : To avoid duplicacy of words (different forms for the same word should have same representation)
#                  its using stem of the word, rather than the word itself
# Use BagOfWords algo for vectorization
# Find n nearest neibhours to find similar movies.
#                : Mind it, we will use Cosine distances between the vectors (Not-Euclidien distances)

In [927]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stemit(text):
    stemmed = []
    for tex in text.split(' '):
        stemmed.append(ps.stem(tex))
    return ' '.join(stemmed)

In [928]:
movies['tags'] = movies['tags'].apply(stemit)

In [929]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english', max_features=5000)

In [930]:
vectors = cv.fit_transform(movies['tags']).toarray()

In [931]:
# movies['tags'][0]
vectors[0].sum()
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [932]:
from sklearn.metrics.pairwise import cosine_similarity
#cosine_similarity(vectors).shape
similarity = cosine_similarity(vectors)
# print similarity matrix
similarity

array([[1.        , 0.13055824, 0.07784989, ..., 0.03109852, 0.        ,
        0.        ],
       [0.13055824, 1.        , 0.0745356 , ..., 0.02977457, 0.        ,
        0.03149704],
       [0.07784989, 0.0745356 , 1.        , ..., 0.02663118, 0.        ,
        0.        ],
       ...,
       [0.03109852, 0.02977457, 0.02663118, ..., 1.        , 0.04397995,
        0.04501491],
       [0.        , 0.        , 0.        , ..., 0.04397995, 1.        ,
        0.09304842],
       [0.        , 0.03149704, 0.        , ..., 0.04501491, 0.09304842,
        1.        ]])

In [939]:
# use enumerate to introduce positional index to every item
# send it to the list and reverse sort it based on the second index
def get_recommendations(movie, count=5):
    recommendations = []

    # right now all the movies have lower and uppercase. Better change them all to lower in the database/dataframe 
    movie = str.lower(movie)

    if np.sum([movies['title'] == movie]) == 0:
        return "Warning: Movie does not exist in our system...!"

    orig_index =  movies[movies['title'] == movie].index[0]
    recommended = sorted(list(enumerate(similarity[orig_index])), reverse=True, key=lambda indx: indx[1])[1:count+1]
    for rec in recommended:
        # Convert first letter to uppercase for convenience
        recommendations.append((movies.loc[rec[0]]['title']).title()) 
    return recommendations

In [941]:
get_recommendations('Jurassic Park', count=20)

['Jurassic World',
 'The Lost World: Jurassic Park',
 'The Nut Job',
 'Moon',
 'Journey To The Center Of The Earth',
 'Land Of The Lost',
 "National Lampoon'S Vacation",
 'Jurassic Park Iii',
 'Stuart Little 2',
 'Vacation',
 'Species',
 'The Love Guru',
 'Nurse 3-D',
 'Clerks Ii',
 'Finishing The Game',
 'Yogi Bear',
 'Walking With Dinosaurs',
 'Scooby-Doo',
 'Broken Arrow',
 'Joe Dirt']

#### The model is in place however, its accuracy and perfromance mesurements are not yet done.
####  I leave it as part of the next exercise.

#### Above all these matrices should be decided in the very beginning. They should be the guiding to the benchmark 